In [ ]:
import numpy as np #import numpy
import matplotlib.pyplot as plt #import matplotlib
from matplotlib.pyplot import rcParams
rcParams['mathtext.default'] = 'regular'
import math
from scipy.interpolate import interp1d, splrep, splev
from pylab import setp
import helper

In [ ]:
from scipy.interpolate import interp1d

energy_GeV, aeffsr_m2sr = helper.get_aeff('ehenextgen')
energy_eV = energy_GeV*1E9
energy_eV = energy_eV[1:]
aeffsr_m2sr = aeffsr_m2sr[1:]

# need interpolator for IceCube sensitivity
icecube_interpolator = interp1d(np.log10(energy_eV), np.log10(aeffsr_m2sr),
                              bounds_error=False,
                              fill_value=0.
                              )

# debug
some_e_bins = np.logspace(15,20,100)
interp_vals = icecube_interpolator(np.log10(some_e_bins))
fig2 = plt.figure(figsize=(5,5))
ax = fig2.add_subplot(1,1,1)
ax.plot(energy_eV, aeffsr_m2sr)
ax.plot(some_e_bins, 10**interp_vals,'--')
plt.loglog()


In [ ]:
# fluxes
def power_law(E_eV, phi, gamma):
    return phi * np.power(E_eV/1.E14, gamma)

# some fluxes (returned in number of particles)
# in units of 10^-23 / eV / m2 / s / sr (note the change in units!)
def icecube_thrumu_function(E_eV):
    return power_law(E_eV, 3.*1.44E-23, -2.37)
def icecube_hese_function(E_eV):
    return power_law(E_eV, 5.68E-23, -2.87)
def icecube_cascade_function(E_eV):
    return power_law(E_eV, 3*1.66e-23, -2.53)

# cosmogenic fluxes
from ehefluxes import fluxes
flux_ahlers20101E18 = fluxes.EHEFlux('cosmogenic_ahlers2010_1E18')

def simple_e_differential_limit(energies, total_aeff_sr, livetime):
        # input 
        # energies (in real units, GeV)
        # flavor averaged effective area (in m2)
        # livetime (in seconds)
        
        # returns
        # E^2 * F(E) (GeV/cm2/s/sr)
        n_90 = 1.
        return n_90 / (livetime * np.log(10) * total_aeff_sr)


In [ ]:
# do an integral
bins = np.arange(14,20,0.5)
livetime = 60*60*24*365*16.56

def run_integral(energy_bins, flux_model, aeff_interp_object):

    energy_bins=[]
    counts = []
    
    for bin in bins:
        temp_logev = np.arange(bin, bin+0.5, 0.01)
        temp_energy = np.power(10., temp_logev)
    
        # the effective area estimate
        temp_aeff = np.power(10., aeff_interp_object(temp_logev))
    
        # the flux estimate
        temp_model = flux_model(temp_energy)
        
        if type(flux_model)==fluxes.EHEFlux:
            temp_model = flux_model(temp_energy/1E9,"sum")/1E9 # for this flux class, need GeV
                
        temp_counts = np.trapz(temp_model*temp_aeff*livetime,temp_energy)
        counts.append(temp_counts)
        energy_bins.append(np.power(10.,bin))

    return energy_bins, counts

energy_bins, counts_thrumu = run_integral(bins, icecube_thrumu_function, icecube_interpolator)
energy_bins, counts_hese = run_integral(bins, icecube_hese_function, icecube_interpolator)
energy_bins, counts_cascade = run_integral(bins, icecube_cascade_function, icecube_interpolator)
energy_bins, counts_ahlers2010 = run_integral(bins, flux_ahlers20101E18, icecube_interpolator)

counts_thrumu = np.array(counts_thrumu)
counts_hese = np.array(counts_hese)
energy_bins = np.array(energy_bins)
counts_ahlers2010 = np.array(counts_ahlers2010)

print(np.sum(counts_thrumu))
print(np.sum(counts_hese))
print(np.sum(counts_cascade))
print(np.sum(counts_ahlers2010))



In [ ]:

xlow =  3.e15 #the lower x limit
xup = 2E20 #the uppper x limit

def beautify_counts(this_ax):
    sizer=25
    this_ax.set_xlabel('Energy [eV]',size=sizer) #give it a title
    this_ax.set_ylabel('Events',size=sizer)
    this_ax.set_xscale('log')
    this_ax.tick_params(labelsize=sizer)
    this_ax.set_xlim([xlow,xup]) #set the x limits of the plot
    this_ax.set_ylim([0,7]) #set the x limits of the plot
    this_legend = this_ax.legend(loc='upper right')#,title='Trigger-Level Event Counts')
    setp(this_legend.get_texts(), fontsize=20)
    setp(this_legend.get_title(), fontsize=20)
    this_legend.get_frame().set_edgecolor('k')
    this_ax.xaxis.set_ticks([1e15, 1e17, 1e19])
    this_ax.yaxis.set_ticks([1,2,3,4,5,6,7])
    # this_ax.grid()

#pass it the axes and the number of theory curves you're including
#always pass the theory curves first
def beautify_limit_withtheory(this_ax):
    sizer=25
    ylow = 1e-20 #the lower y limit
    yup = 1e-13 #the upper y limit
    this_ax.set_xlabel('Energy [eV]',size=sizer) #give it a title
    this_ax.set_ylabel('E F(E) [$cm^{-2} s^{-1} sr^{-1}$]',size=sizer)
    this_ax.set_yscale('log')
    this_ax.set_xscale('log')
    this_ax.tick_params(labelsize=sizer)
    this_ax.set_xlim([xlow,xup]) #set the x limits of the plot
    this_ax.set_ylim([ylow,yup]) #set the y limits of the plot
    this_ax.yaxis.set_ticks([1e-18, 1e-16, 1e-14])
    this_ax.xaxis.set_ticks([1e15, 1e17, 1e19])
    # this_ax.grid()

In [ ]:
fig2 = plt.figure(figsize=(2*8.5,2*3.5))
ax2_lim = fig2.add_subplot(1,2,1)
ax2_counts = fig2.add_subplot(1,2,2)
plot_bins = np.power(10., np.linspace(13,22,20))
# plot_bins = np.power(10.,np.arange(13,22,1))

plt.rc('text',usetex=True) #plot with Latex
plt.rc('font',family='serif') #Used a serif font
plt.rc('font', weight='heavy') #Make the text bold


icecube_energy_logev = np.linspace(14,21,14)
icecube_energy = np.power(10.,icecube_energy_logev)
icecube_thrumu_e2fe = icecube_thrumu_function(icecube_energy) * icecube_energy
icecube_hese_e2fe = icecube_hese_function(icecube_energy) * icecube_energy
icecube_ahlers_e2fe = flux_ahlers20101E18(icecube_energy/1E9, "sum") * icecube_energy/1E9

# models
line1, = ax2_lim.plot(icecube_energy,icecube_thrumu_e2fe,label=r'IceCube Thru-Mu 2022 (E$^{-2.19}$)',
                      color='C0', linewidth=4,linestyle='-',alpha=0.7)
line2, = ax2_lim.plot(icecube_energy,icecube_hese_e2fe,'-',label=r'IceCube HESE 2021 (E$^{-2.89}$)',
                      color='C1', linewidth=4,linestyle='--',alpha=0.7)
line3, = ax2_lim.plot(icecube_energy,icecube_ahlers_e2fe,'-', label=r'Ahlers 2010',
                      color='C2', linewidth=4,linestyle='-.',alpha=0.7)

# limit
icecube_aeff = np.power(10.,icecube_interpolator(icecube_energy_logev))
lim, = ax2_lim.plot(icecube_energy,
        simple_e_differential_limit(icecube_energy, icecube_aeff,livetime),
        linewidth=5., color='black', linestyle='-', markersize=12,
        label='New EHE 12yrs, sensitivity')

beautify_limit_withtheory(ax2_lim)
legend1 = ax2_lim.legend(handles=[line1, line2, line3], 
                         labels=['IceCube Thru-Mu 2016 (E$^{-2.37}$)',r'IceCube HESE 2021 (E$^{-2.89}$)','Ahlers 2010'], 
                         loc='lower left',title='Flux Models',facecolor='white',framealpha=1)
ax2_lim.add_artist(legend1)
legend2 = ax2_lim.legend(handles=[lim], labels=['Projected IceCube Sensitivity'], loc='upper right')
setp(legend1.get_texts(), fontsize=20)
setp(legend1.get_title(), fontsize=20)
setp(legend2.get_texts(), fontsize=20)
setp(legend2.get_title(), fontsize=20)
legend1.get_frame().set_edgecolor('k')
legend2.get_frame().set_edgecolor('k')


# event counts
n, bins, patches= ax2_counts.hist(energy_bins, bins=plot_bins, weights=counts_thrumu,
                                    label=r'Thru-Mu 2022 ($E^{-2.37}$): %.1f events'%counts_thrumu.sum(),
                                    fill=False, histtype='step', 
                                    edgecolor='C0', linewidth=4,linestyle='-')
n, bins, patches= ax2_counts.hist(energy_bins, bins=plot_bins, weights=counts_hese,
                                    label=r'HESE 2021 ($E^{-2.89}$): %.1f events'%counts_hese.sum(),
                                    fill=False,  histtype='step',  
                                    edgecolor='C1', linewidth=4,linestyle='--')
n, bins, patches= ax2_counts.hist(energy_bins,
                                    bins=plot_bins,
                                    weights=counts_ahlers2010,
                                    label=r'Ahlers 2010: %.1f events'%counts_ahlers2010.sum(),
                                    fill=False,  histtype='step',  
                                    edgecolor='C2', linewidth=4, linestyle='-.')

beautify_counts(ax2_counts)

fig2.savefig("limit_and_counts.pdf",edgecolor='none',bbox_inches="tight",dpi=300) #save the figure    